In [1]:
from cobra import Metabolite, Reaction
import json


def df_to_reaction(df, model):
    """

    :param df: DataFrame with two columns.
    The first column contains metabolite IDs,
    the second column contains coefficients.
    :param model: The model that contains the metabolite objects
    :return: Returns a Reaction that contains Metabolite objects
    (gathered from model by using the metabolite IDs from df),
    with the corresponding coefficients from df.
    """

    metabolite_to_coefficient = {}  # dictionary which will be used to map from Metabolite object to coefficient (float)
    for list in df.values.tolist():
        metabolite_to_coefficient[model.metabolites.get_by_id(list[1])] = float(list[0])

    reaction = Reaction()

    reaction.add_metabolites(metabolite_to_coefficient)

    return reaction


def add_attributes(reaction, reaction_id, model):
    """

    :param reaction: Reaction object.
    :param reaction_id: Reaction ID of the reaction that we want the attributes from.
    :param model: Model which contains Reaction object with Reaction ID.
    :return: Reaction object with the same id, name, subsystem, lower- and upper bound as
    Reaction object from model with id = reaction_id
    """
    reaction.id = model.reactions.get_by_id(reaction_id).id + "_new"
    reaction.name = model.reactions.get_by_id(reaction_id).name
    reaction.subsystem = model.reactions.get_by_id(reaction_id).subsystem
    reaction.lower_bound = model.reactions.get_by_id(reaction_id).lower_bound
    reaction.upper_bound = model.reactions.get_by_id(reaction_id).upper_bound

    return reaction


def reversibility_check(reaction):
    """

    :param reaction: Reaction object.
    :return: lower and upper flux bounds of Reaction object.
    """
    return reaction.lower_bound, reaction.upper_bound


def new_synthesis_reaction(df, rxn_id, model):
    """

    :param df: DataFrame with two columns: Coefficient (mmol/gDW) and metabolite ID.
    :param rxn_id: Reaction ID of the original synthesis Reaction object.
    :param model: The metabolic model in use.
    :return: New synthesis Reaction object with the metabolites and coefficients from df.
    """
    synthesis_reaction = df_to_reaction(df, model)
    synthesis_reaction = add_attributes(synthesis_reaction, rxn_id, model)

    return synthesis_reaction


def set_bounds(model, solution):
    for exchange_ID in solution.keys():

        if solution[exchange_ID] < 0:

            model.reactions.get_by_id(exchange_ID).lower_bound = solution[exchange_ID]
            model.reactions.get_by_id(exchange_ID).upper_bound = 0.0

        else:

            model.reactions.get_by_id(exchange_ID).upper_bound = solution[exchange_ID]
            model.reactions.get_by_id(exchange_ID).lower_bound = 0.0


def write_flux_json(filename, model):
    """ write_flux_json writes a json file from a dictionary of model reactions
    and associated flux values
    """
    flux_dict = model.optimize().fluxes.to_dict()  # reaction:flux dict
    with open(filename, "w") as f:
        json.dump(flux_dict, f)


def write_min_flux_json(filename, fva_solution):
    """ write_min_flux_json writes json files from the minimal flux-values found with fva
    """
    min_flux_dict = fva_solution["minimum"].to_dict()
    with open(filename, "w") as f:
        json.dump(min_flux_dict, f)


def write_max_flux_json(filename, fva_solution):
    """ write_max_flux_json writes json files from the maximal flux-values found with fva
    """
    max_flux_dict = fva_solution["maximum"].to_dict()
    with open(filename, "w") as f:
        json.dump(max_flux_dict, f)
